# Guideline
*Arthur Ryman, last updated 2025-03-23*

## Introduction

The purpose of this notebook is to develop Python code that implements the 
[Guideline](https://www.fct-cf.gc.ca/en/pages/representing-yourself/deadlines-calculator/guideline)
for calculating Federal Court due dates.

## Federal Court Rules

The following provisions of the 
[Federal Courts Rules](https://laws-lois.justice.gc.ca/eng/regulations/sor-98-106/FullText.html)
will assist you in computing the time within which to file and serve your documents.

* Rule 6(1) incorporates the provisions of sections 26 to 30 of the Interpretation Act for computing time.
* Rule 6(2) governs the computation of periods of less than 7 days.
* Rule 6(3) addresses the exclusion of the seasonal recess in the computation of time provided by the Rules.
* Rules 7 and 8 govern extensions by consent and the Court’s discretion to extend or abridge time periods.
* Holiday means a Saturday, Sunday or any other day defined as a holiday in subsection 35(1) of the Interpretation Act.
* Summer recess means the months of July and August in each year.
* Seasonal recess means the period beginning on December 21 in a year and ending on January 7 in the following year.

## Deadlines - General

> When a document is to be filed or served within a defined number of days before or after a specified event, 
do not include the day of the event when calculating the deadline, but do include all other days, 
including weekends and holidays (please see exceptions). The deadline includes the last day.

At a high level, the problem is to compute a deadline for filing or serving a document.
We'll generically refer to filing or serving a document as an *action*.
The deadline for the action is triggered by some *event*.

The deadline may be either before or after the event.

For example, the event might be a court ruling and the action might be filing an appeal.
In this case, the deadline is triggered by the date of the court ruling and comes after it.
Presumably, an appeal filed after the deadline would be rejected.

On the other hand, the event might be an upcoming trial in which case the parties may have to file evidence in
advance of the event. 
In this case the deadline is before the event.

Our task is to implement a function that computes the deadline date given the following input data:
* `event_date`: the date of the event
* `number_of_days`: the maximum permissible number of days that can elapse between the event and the required action
We'll have to identity the set of distinct action types and implement the policies associated with each.
We can start with *general* actions.

Next, we consider the examples provided in the Guideline document.

### Example 1 - 10 Days

To calculate a 10-day deadline after a specific event, 
for example the filing of a document, do not include the first day of the period but do include the 10th day.  
In this example, the period starts on May 31st and the 10th day is a Sunday, so the document is due on the next business day.

<figure style="text-align: center;">
    <img src="images/guideline/general-example-1.png">
    <figcaption>Example 1 - 10 Days</figcaption>
</figure>

> The months of May (31 days) and June (30 days) are represented side by side in a calendar view. 
Thursday May 31st and Monday June 11th are highlighted in yellow to show the 10-day period taking into account the weekend.

#### What year are the example calendars taken from?

We'll use the examples given the Guideline document as test cases for our code.
However, the calendars do not indicate the year.
We need to determine the year so that we can exactly match the examples.

The calendar in Example 1 shows May 1 on a Tuesday.
Find the most recent year as of 2024 in which May 1 was a Tuesday. 

In [1]:
import calendar
from deadlines.dates import find_year

find_year(calendar.MAY, 1, calendar.TUESDAY, 2024)

2018

Verify this result by printing the calendar for May 2018.

In [2]:
!python -m calendar -f 6 2018 5

      May 2018
Su Mo Tu We Th Fr Sa
       1  2  3  4  5
 6  7  8  9 10 11 12
13 14 15 16 17 18 19
20 21 22 23 24 25 26
27 28 29 30 31


The calendar for May 2018 matches the one shown in Example 1.

#### Example 1

The event date is 2018-05-31 and the number of days is 10.
The correct deadline date is 2018-06-11.

In [3]:
import datetime

event_date = datetime.date(2018, calendar.MAY, 31)
number_of_days = 10
deadline_date = datetime.date(2018, calendar.JUNE, 11)

event_date, deadline_date

(datetime.date(2018, 5, 31), datetime.date(2018, 6, 11))

To compute the due date we first find the date that is 10 days after the event date.
We DO NOT include the day of the event but do include weekends and holidays. However, there are some exceptions
which we'll ignore at the moment. We do include the last day as part of the 10-day period.
However, if the last day falls on a weekend then the deadline is the next business day.

I assume that in general the deadline MUST be a business day.
Here I assume that a business day means that the court is open.
This excludes weekends, holidays, and recesses.
For now just consider weekends. 
We'll need to get a precise definition of holidays since this varies by jurisdiction.

In [4]:
from deadlines.due_dates import deadline_after

deadline_after(event_date, number_of_days)

datetime.date(2018, 6, 11)

In [5]:
deadline_date == deadline_after(event_date, number_of_days)

True

Our simple code computes the right answer.

### Example 2 - 20 Days

To calculate a 20-day deadline after a specific event, for example the filing of a document, 
do not include the first day of the period but do include the 20th day. 
In this example, the period starts on May 1st and the 20th day is a holiday, 
so the document is due on the next business day, which is Tuesday May 22nd.

<figure style="text-align: center;">
    <img src="images/guideline/general-example-2.png">
    <figcaption>Example 2 - 20 Days</figcaption>
</figure>

> Wednesday May 1st and Tuesday May 22nd are highlighted in yellow to show the 20-day period.

> Note: in this example May 21st, which is a holiday (Victoria Day) is calculated in the computation of time.

In [6]:
event_date = datetime.date(2018, calendar.MAY, 1)
number_of_days = 20
deadline_date = datetime.date(2018, calendar.MAY, 22)

deadline_after(event_date, number_of_days)

datetime.date(2018, 5, 21)

Now we get the wrong answer because we haven't considered holidays.
We need a list of holidays. This is a complication because some holidays depend on the jurisdiction.

Where are the holidays defined?
The Federal Court Guideline page contains the following statement:

> Holiday means a Saturday, Sunday or any other day defined as a holiday in
> [subsection 35(1)](https://laws-lois.justice.gc.ca/eng/acts/I-21/page-3.html#h-279462) of the
> [Interpretation Act](https://laws-lois.justice.gc.ca/eng/acts/I-21/).

Here is the offical definition of *holiday*.

**holiday** means any of the following days, namely, 
Sunday; New Year’s Day; Good Friday; Easter Monday; Christmas Day; 
the birthday or the day fixed by proclamation for the celebration of the birthday of the reigning Sovereign; 
Victoria Day; Canada Day; the first Monday in September, designated Labour Day; 
National Day for Truth and Reconciliation, which is observed on September 30; Remembrance Day; 
any day appointed by proclamation to be observed as a day of general prayer or mourning or day of public rejoicing or thanksgiving; 
and any of the following additional days, namely,

(a) in any province, any day appointed by proclamation of the lieutenant governor of the province to be observed as a public holiday 
or as a day of general prayer or mourning or day of public rejoicing or thanksgiving within the province, 
and any day that is a non-juridical day by virtue of an Act of the legislature of the province, and

(b) in any city, town, municipality or other organized district, 
any day appointed to be observed as a civic holiday by resolution of the council or other authority charged with the 
administration of the civic or municipal affairs of the city, town, municipality or district; (jour férié)

We'll start by implementing the definitions of the Federal holidays.
As for the provincial and local holidays, we only need to define those for the locations of Federal courts.
I assume that there is a list of the relevant courts somewhere.

The following holidays are defined in the [Holidays Act](https://laws-lois.justice.gc.ca/eng/acts/h-5/page-1.html).

```
Holidays Act
R.S.C., 1985, c. H-5

An Act respecting Holidays

Short Title
Marginal note:Short title

1 This Act may be cited as the Holidays Act.

R.S., c. H-7, s. 1
Canada Day
Marginal note:Canada Day

2 (1) July 1, not being a Sunday, is a legal holiday and shall be kept and observed as such throughout Canada under the name of "Canada Day".

Marginal note:When July 1 is a Sunday

(2) When July 1 is a Sunday, July 2 is a legal holiday and shall be kept and observed as such throughout Canada under the name of "Canada Day".

R.S., c. H-7, s. 21980-81-82-83, c. 124, s. 1
Remembrance Day
Marginal note:Remembrance Day

3 November 11, being the day in the year 1918 on which the Great War was triumphantly concluded by an armistice, is a legal holiday and shall be kept and observed as such throughout Canada under the name of “Remembrance Day”.

R.S., 1985, c. H-5, s. 32018, c. 3, s. 1
Previous Version
Victoria Day
Marginal note:Victoria Day

4 The first Monday immediately preceding May 25 is a legal holiday and shall be kept and observed as such throughout Canada under the name of "Victoria Day".

R.S., c. H-7, s. 4
```

There is a Python package that claims to compute holidays for many countries.
See: https://pypi.org/project/holidays/

In [7]:
new_years_day = datetime.date(2025, calendar.Month.JANUARY, 1)
national_day = datetime.date(2025, calendar.Month.SEPTEMBER, 30)
christmas_day = datetime.date(2025, calendar.Month.DECEMBER, 25)

new_years_day

datetime.date(2025, 1, 1)

In [8]:
def is_new_years_day(date: datetime.date) -> bool:
    # New Year's Day is January 1
    return date.month == calendar.Month.JANUARY and date.day == 1

is_new_years_day(new_years_day), is_new_years_day(national_day), is_new_years_day(christmas_day)

(True, False, False)